# Neighbourhoods in Mumbai

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
colors_old = { 'Western Suburbs':'red','Eastern Suburbs':'blue','Harbour Suburbs':'purple','South Mumbai':'green',}
df=pd.read_excel('Neighbourhoods_in_Mumbai.xlsx')
df

,Suburbs,Neighborhood,Suburbs_Zone,Latitude,Longitude
0,Andheri,Andheri,Western Suburbs,19.119000,72.847000
1,Andheri,Marol,Western Suburbs,19.117300,72.884000
2,Bhayandar,Bhayandar,Western Suburbs,19.290000,72.850000
3,Bandra,Bandra,Western Suburbs,19.054444,72.840556
4,Borivali (West),Borivali (West),Western Suburbs,19.238100,72.852300
5,Borivali (East),Borivali (East),Western Suburbs,19.229800,72.860900
6,Dahisar,Dahisar,Western Suburbs,19.250069,72.859347
7,Goregaon,Goregaon,Western Suburbs,19.155000,72.850000
8,Jogeshwari,Jogeshwari,Western Suburbs,19.143900,72.842800
9,Juhu,Juhu,Western Suburbs,19.100000,72.830000


In [3]:
address = 'Mumbai Suburban'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

19.13095765 72.88593095460952


In [4]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, label, zone in zip(df.Latitude, df.Longitude, df.Neighborhood, df.Suburbs_Zone):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color=colors_old[zone],
        popup=label,
        fill = True,
        fill_color=colors_old[zone],
        fill_opacity=0.6
    ).add_to(venues_map)
    
venues_map

#### Define Foursquare Credentials and Version

In [5]:
CLIENT_ID = 'WZXGGFYR4BVJAFMNNNS2QBSQBPHNDRW2TWAVESILDLYCW552' # your Foursquare ID
CLIENT_SECRET = 'XGNR3QG4HZGSA5QBU0FCKFUJFD5DQXHPWROG14KOQSG1DKKH' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WZXGGFYR4BVJAFMNNNS2QBSQBPHNDRW2TWAVESILDLYCW552
CLIENT_SECRET:XGNR3QG4HZGSA5QBU0FCKFUJFD5DQXHPWROG14KOQSG1DKKH


# Explore Neighborhoods in Mumbai

#### Let's create a function to repeat the same process to all the neighborhoods.

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100 # limit of number of venues returned by Foursquare API
    radius = 500 # define radius
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [7]:
mumbai_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Andheri
Marol
Bhayandar
Bandra
Borivali (West)
Borivali (East)
Dahisar
Goregaon
Jogeshwari
Juhu
Kandivali west
Kandivali east
Khar
Malad
Santacruz
Vasai
Naigaon
Nalasopara
Virar
Vile Parle
Bhandup
Ghatkopar
Kanjurmarg
Kurla
Mulund
Powai
Vidyavihar
Vikhroli
Chembur
Govandi
Mankhurd
Trombay
Agripada
Churchgate
Cotton Green
Cuffe Parade
Cumbala Hill
Dhobitalao
Dongri
Kala Ghoda
Lower Parel
Mahalaxmi
Mahim
Malabar Hill
Marine Lines
Mumbai Central
Nariman Point
Prabhadevi
Sion
Worli
Dharavi
Koliwada
Kumbarwara
Antop Hill
Byculla
Colaba
Dadar
Fort
Girgaon
Kalbadevi
Kamathipura
Parel
Tardeo


#### Let's check the size of the resulting dataframe

In [8]:
print(mumbai_venues.shape)
type(mumbai_venues)

(906, 7)


pandas.core.frame.DataFrame

Let's check how many venues were returned for each neighborhood

In [9]:
mumbai_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agripada,4,4,4,4,4,4
Andheri,12,12,12,12,12,12
Antop Hill,6,6,6,6,6,6
Bandra,19,19,19,19,19,19
Bhandup,4,4,4,4,4,4
Bhayandar,1,1,1,1,1,1
Borivali (East),7,7,7,7,7,7
Borivali (West),9,9,9,9,9,9
Byculla,10,10,10,10,10,10


#### Let's find out how many unique categories can be curated from all the returned venues

In [10]:
print('There are {} uniques categories.'.format(len(mumbai_venues['Venue Category'].unique())))

There are 159 uniques categories.


# Analyze Each Neighborhood

In [11]:
# one hot encoding
mumbai_onehot = pd.get_dummies(mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Neighborhood'] = mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

mumbai_onehot.head()

,Neighborhood,ATM,Accessories Store,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Station,Café,Cantonese Restaurant,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field,Flea Market,Flower Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Goan Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lighthouse,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Mountain,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Nightclub,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Restaurant,Roof Deck,Sake Bar,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tex-Mex Restaurant,Theater,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Andheri,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [12]:
mumbai_onehot.shape

(906, 160)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [13]:
mumbai_grouped = mumbai_onehot.groupby('Neighborhood').mean().reset_index()
mumbai_grouped

,Neighborhood,ATM,Accessories Store,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Station,Café,Cantonese Restaurant,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Field,Flea Market,Flower Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Goan Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lighthouse,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Mountain,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Nightclub,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Restaurant,Roof Deck,Sake Bar,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tex-Mex Restaurant,Theater,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Agripada,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.0
1,Andheri,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.166667,0.000000,0.000000,0.000000,0.000000,0.08333

#### Let's confirm the new size

In [14]:
mumbai_grouped.shape

(62, 160)

#### Let's print each neighborhood along with the top 5 most common venues

In [15]:
num_top_venues = 5

for hood in mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = mumbai_grouped[mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agripada----
               venue  freq
0  Indian Restaurant  0.25
1        Coffee Shop  0.25
2             Bakery  0.25
3                Gym  0.25
4                ATM  0.00


----Andheri----
                  venue  freq
0  Fast Food Restaurant  0.17
1     Indian Restaurant  0.17
2        Ice Cream Shop  0.08
3                  Café  0.08
4            Food Truck  0.08


----Antop Hill----
                  venue  freq
0           IT Services  0.17
1                 Trail  0.17
2         Grocery Store  0.17
3  Gym / Fitness Center  0.17
4     Indian Restaurant  0.17


----Bandra----
                venue  freq
0   Indian Restaurant  0.26
1                Café  0.11
2  Italian Restaurant  0.05
3      Breakfast Spot  0.05
4                Pier  0.05


----Bhandup----
                     venue  freq
0        Indian Restaurant  0.50
1     Gym / Fitness Center  0.25
2               Restaurant  0.25
3         Parsi Restaurant  0.00
4  North Indian Restaurant  0.00


----Bhayandar----
 

4               Chaat Place  0.06


----Parel----
               venue  freq
0  Indian Restaurant  0.25
1              Plaza  0.25
2         Whisky Bar  0.25
3   Asian Restaurant  0.25
4             Office  0.00


----Powai----
                venue  freq
0   Indian Restaurant  0.14
1                Café  0.06
2  Chinese Restaurant  0.04
3          Restaurant  0.04
4  Italian Restaurant  0.04


----Prabhadevi----
               venue  freq
0  Indian Restaurant  0.20
1  Electronics Store  0.10
2               Café  0.10
3         Smoke Shop  0.05
4        Snack Place  0.05


----Santacruz----
               venue  freq
0  Indian Restaurant  0.17
1         Steakhouse  0.06
2      Jewelry Store  0.06
3             Market  0.06
4                Gym  0.06


----Sion----
                           venue  freq
0              Indian Restaurant  0.25
1  Vegetarian / Vegan Restaurant  0.17
2           Gym / Fitness Center  0.17
3                            Gym  0.08
4                    Snack Pl

#### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mumbai_grouped['Neighborhood']

for ind in np.arange(mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agripada,Bakery,Indian Restaurant,Coffee Shop,Gym,Donut Shop,Food,Flower Shop,Flea Market,Field,Fast Food Restaurant
1,Andheri,Indian Restaurant,Fast Food Restaurant,Bakery,Electronics Store,Food Truck,Café,Sandwich Place,Restaurant,Food Court,Ice Cream Shop
2,Antop Hill,IT Services,Indian Restaurant,Bus Station,Gym / Fitness Center,Trail,Grocery Store,Diner,Flea Market,Field,Fast Food Restaurant
3,Bandra,Indian Restaurant,Café,Brewery,Furniture / Home Store,Convenience Store,Pier,Design Studio,Platform,Dessert Shop,Restaurant
4,Bhandup,Indian Restaurant,Restaurant,Gym / Fitness Center,Zoo,Flower Shop,Flea Market,Field,Fast Food Restaurant,Electronics Store,Dumpling Restaurant


# Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [18]:
# set number of clusters
kclusters = 5

mumbai_grouped_clustering = mumbai_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 4, 1, 1, 1, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [19]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mumbai_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mumbai_merged = mumbai_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



# check the last columns!
mumbai_merged#.head()

,Suburbs,Neighborhood,Suburbs_Zone,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Andheri,Andheri,Western Suburbs,19.119000,72.847000,1.0,Indian Restaurant,Fast Food Restaurant,Bakery,Electronics Store,Food Truck,Café,Sandwich Place,Restaurant,Food Court,Ice Cream Shop
1,Andheri,Marol,Western Suburbs,19.117300,72.884000,1.0,Indian Restaurant,Snack Place,Food,Bus Station,Diner,Hotel,Ice Cream Shop,Cupcake Shop,Electronics Store,Flower Shop
2,Bhayandar,Bhayandar,Western Suburbs,19.290000,72.850000,4.0,Shipping Store,Zoo,Cosmetics Shop,Food,Flower Shop,Flea Market,Field,Fast Food Restaurant,Electronics Store,Dumpling Restaurant
3,Bandra,Bandra,Western Suburbs,19.054444,72.840556,1.0,Indian Restaurant,Café,Brewery,Furniture / Home Store,Convenience Store,Pier,Design Studio,Platform,Dessert Shop,Restaurant
4,Borivali (West),Borivali (West),Western Suburbs,19.238100,72.852300,1.0,Pharmacy,Indian Restaurant,Pizza Place,Garden Center,Italian Restaurant,Gym,Snack Place,Ice Cream Shop,Park,Gourmet Shop
5,Borivali (East),Borivali (East),Western Suburbs,19.229800,72.860900,1.0,Snack Place,Burger Joint,Clothing Store,Chinese Restaurant,Scenic Lookout,Park,Indian Restaurant,Harbor / Marina,Field,Fast Food Restaurant
6,Dahisar,Dahisar,Western Suburbs,19.250069,72.859347,0.0,Movie Theater,Pizza Place,Indian Restaurant,Café,Train Station,Restaurant,Bar,Zoo,Dumpling Restaurant,Flea Market
7,Goregaon,Goregaon,Western Suburbs,19.155000,72.850000,0.0,Sandwich Place,Coffee Shop,Train Station,Zoo,Donut Shop,Flower Shop,Flea Market,Field,Fast Food Restaurant,Electronics Store
8,Jogeshwari,Jogeshwari,Western Suburbs,19.143900,72.842800,0.0,Ice Cream Shop,Men's Store,Café,Zoo,Dumpling Restaurant,Food,Flower Shop,Flea Market,Field,Fast Food Restaurant
9,Juhu,Juhu,Western Suburbs,19.100000,72.830000,1.0,Indian Restaurant,Bar,Hotel,Café,Clothing Store,Spa,Cantonese Restaurant,Boutique,Lounge,Food Truck


Finally, let's visualize the resulting clusters

In [20]:
mumbai_merged=mumbai_merged.dropna()
#int(mumbai_merged['Cluster Labels'][0])

In [21]:
mumbai_merged

,Suburbs,Neighborhood,Suburbs_Zone,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Andheri,Andheri,Western Suburbs,19.119000,72.847000,1.0,Indian Restaurant,Fast Food Restaurant,Bakery,Electronics Store,Food Truck,Café,Sandwich Place,Restaurant,Food Court,Ice Cream Shop
1,Andheri,Marol,Western Suburbs,19.117300,72.884000,1.0,Indian Restaurant,Snack Place,Food,Bus Station,Diner,Hotel,Ice Cream Shop,Cupcake Shop,Electronics Store,Flower Shop
2,Bhayandar,Bhayandar,Western Suburbs,19.290000,72.850000,4.0,Shipping Store,Zoo,Cosmetics Shop,Food,Flower Shop,Flea Market,Field,Fast Food Restaurant,Electronics Store,Dumpling Restaurant
3,Bandra,Bandra,Western Suburbs,19.054444,72.840556,1.0,Indian Restaurant,Café,Brewery,Furniture / Home Store,Convenience Store,Pier,Design Studio,Platform,Dessert Shop,Restaurant
4,Borivali (West),Borivali (West),Western Suburbs,19.238100,72.852300,1.0,Pharmacy,Indian Restaurant,Pizza Place,Garden Center,Italian Restaurant,Gym,Snack Place,Ice Cream Shop,Park,Gourmet Shop
5,Borivali (East),Borivali (East),Western Suburbs,19.229800,72.860900,1.0,Snack Place,Burger Joint,Clothing Store,Chinese Restaurant,Scenic Lookout,Park,Indian Restaurant,Harbor / Marina,Field,Fast Food Restaurant
6,Dahisar,Dahisar,Western Suburbs,19.250069,72.859347,0.0,Movie Theater,Pizza Place,Indian Restaurant,Café,Train Station,Restaurant,Bar,Zoo,Dumpling Restaurant,Flea Market
7,Goregaon,Goregaon,Western Suburbs,19.155000,72.850000,0.0,Sandwich Place,Coffee Shop,Train Station,Zoo,Donut Shop,Flower Shop,Flea Market,Field,Fast Food Restaurant,Electronics Store
8,Jogeshwari,Jogeshwari,Western Suburbs,19.143900,72.842800,0.0,Ice Cream Shop,Men's Store,Café,Zoo,Dumpling Restaurant,Food,Flower Shop,Flea Market,Field,Fast Food Restaurant
9,Juhu,Juhu,Western Suburbs,19.100000,72.830000,1.0,Indian Restaurant,Bar,Hotel,Café,Clothing Store,Spa,Cantonese Restaurant,Boutique,Lounge,Food Truck


In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['Latitude'], mumbai_merged['Longitude'], mumbai_merged['Neighborhood'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    colorid=int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[colorid],
        fill=True,
        fill_color=rainbow[colorid],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

### Cluster 1

In [23]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 0, mumbai_merged.columns[[1] + list(range(5, mumbai_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Dahisar,0.0,Movie Theater,Pizza Place,Indian Restaurant,Café,Train Station,Restaurant,Bar,Zoo,Dumpling Restaurant,Flea Market
7,Goregaon,0.0,Sandwich Place,Coffee Shop,Train Station,Zoo,Donut Shop,Flower Shop,Flea Market,Field,Fast Food Restaurant,Electronics Store
8,Jogeshwari,0.0,Ice Cream Shop,Men's Store,Café,Zoo,Dumpling Restaurant,Food,Flower Shop,Flea Market,Field,Fast Food Restaurant
10,Kandivali west,0.0,Food Truck,Restaurant,Fast Food Restaurant,Bar,Food,Flower Shop,Flea Market,Field,Electronics Store,Dumpling Restaurant
11,Kandivali east,0.0,Ice Cream Shop,Coffee Shop,Bakery,Smoke Shop,Shopping Mall,Sandwich Place,Indian Restaurant,Soccer Field,Dessert Shop,Plaza
16,Naigaon,0.0,Platform,Train Station,Gym,Zoo,Dhaba,Flea Market,Field,Fast Food Restaurant,Electronics Store,Dumpling Restaurant
17,Nalasopara,0.0,Multiplex,Ice Cream Shop,Fast Food Restaurant,Zoo,Donut Shop,Flower Shop,Flea Market,Field,Electronics Store,Dumpling Restaurant
18,Virar,0.0,Pizza Place,Food Stand,Platform,Zoo,Diner,Flea Market,Field,Fast Food Restaurant,Electronics Store,Dumpling Restaurant
23,Kurla,0.0,IT Services,Music Venue,Jewelry Store,Gift Shop,Donut Shop,Flower Shop,Flea Market,Field,Fast Food Restaurant,Electronics Store
24,Mulund,0.0,Café,Ice Cream Shop,Indian Restaurant,Restaurant,Flea Market,Snack Place,Train Station,Gym,Pizza Place,Asian Restaurant


### Cluster 2

In [24]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 1, mumbai_merged.columns[[1] + list(range(5, mumbai_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Andheri,1.0,Indian Restaurant,Fast Food Restaurant,Bakery,Electronics Store,Food Truck,Café,Sandwich Place,Restaurant,Food Court,Ice Cream Shop
1,Marol,1.0,Indian Restaurant,Snack Place,Food,Bus Station,Diner,Hotel,Ice Cream Shop,Cupcake Shop,Electronics Store,Flower Shop
3,Bandra,1.0,Indian Restaurant,Café,Brewery,Furniture / Home Store,Convenience Store,Pier,Design Studio,Platform,Dessert Shop,Restaurant
4,Borivali (West),1.0,Pharmacy,Indian Restaurant,Pizza Place,Garden Center,Italian Restaurant,Gym,Snack Place,Ice Cream Shop,Park,Gourmet Shop
5,Borivali (East),1.0,Snack Place,Burger Joint,Clothing Store,Chinese Restaurant,Scenic Lookout,Park,Indian Restaurant,Harbor / Marina,Field,Fast Food Restaurant
9,Juhu,1.0,Indian Restaurant,Bar,Hotel,Café,Clothing Store,Spa,Cantonese Restaurant,Boutique,Lounge,Food Truck
12,Khar,1.0,Indian Restaurant,Bar,Pub,Lounge,Gastropub,Restaurant,Fast Food Restaurant,Bengali Restaurant,Beer Garden,Beer Bar
13,Malad,1.0,Snack Place,Indian Restaurant,Dessert Shop,Hotel Bar,Train,Fast Food Restaurant,Ice Cream Shop,Cricket Ground,Dumpling Restaurant,Food
14,Santacruz,1.0,Indian Restaurant,Market,Clothing Store,Platform,Chinese Restaurant,Steakhouse,Snack Place,Bakery,Moving Target,Jewelry Store
19,Vile Parle,1.0,Hotel,Café,Indian Restaurant,Sandwich Place,Playground,Cupcake Shop,Pizza Place,Nightclub,Coffee Shop,Cocktail Bar


### Cluster 3

In [25]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 2, mumbai_merged.columns[[1] + list(range(5, mumbai_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,Cuffe Parade,2.0,Pharmacy,Garden,Food Court,Flower Shop,Flea Market,Field,Fast Food Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop
55,Colaba,2.0,Pharmacy,Garden,Food Court,Flower Shop,Flea Market,Field,Fast Food Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop


### Cluster 4

In [26]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 3, mumbai_merged.columns[[1] + list(range(5, mumbai_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Vasai,3.0,Chinese Restaurant,Cosmetics Shop,Food,Flower Shop,Flea Market,Field,Fast Food Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop


### Cluster 5

In [27]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 4, mumbai_merged.columns[[1] + list(range(5, mumbai_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Bhayandar,4.0,Shipping Store,Zoo,Cosmetics Shop,Food,Flower Shop,Flea Market,Field,Fast Food Restaurant,Electronics Store,Dumpling Restaurant
